# Retraso en la llegada de datos

In [ ]:
import org.apache.spark._

In [ ]:
import org.apache.spark.streaming._

In [ ]:
import org.apache.spark.sql.streaming._

In [ ]:
import org.apache.spark.sql.SparkSession

In [ ]:
import org.apache.spark.sql.types._

In [ ]:
import org.apache.spark.sql.Row

In [ ]:
import org.apache.spark.sql.functions._

In [ ]:
val spark = SparkSession.builder.getOrCreate()

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", "10")

In [ ]:
val datos_no_format = spark.readStream.format("socket").
option("host","localhost").option("port", 19002).load()

In [ ]:
val datos_splitted = datos_no_format.withColumn("tmp", split($"value", ";"))

In [ ]:
import java.sql.Timestamp

In [ ]:
val datos = datos_splitted.select(from_unixtime($"tmp".getItem(0).cast(LongType) / 1000).cast(TimestampType).as("ts"), 
                                  $"tmp".getItem(1).as("nodo"),
                                  $"tmp".getItem(2).as("sensor"),
                                  $"tmp".getItem(3).as("valor"))

## Problemática con el retraso de los datos

In [ ]:
val aux_query_1 = datos.groupBy(window($"ts", "45 seconds", "15 seconds"), $"sensor").
count()

In [ ]:
val query_1 = aux_query_1.writeStream.format("console").outputMode("complete").queryName("query 1").
option("truncate", false).
trigger(Trigger.ProcessingTime("30 seconds")).start()

In [ ]:
query_1.stop

# `Watermark`

In [ ]:
val aux_query_2 = datos.withWatermark("ts", "20 seconds").
groupBy(window($"ts", "45 seconds", "15 seconds"), $"sensor").count()

In [ ]:
val query_2 = aux_query_2.writeStream.format("console").outputMode("complete").queryName("query 2").
option("truncate", false).
trigger(Trigger.ProcessingTime("30 seconds")).start()

In [ ]:
val aux_query_3 = datos.withWatermark("ts", "20 seconds").
groupBy(window($"ts", "45 seconds", "15 seconds"), $"sensor").count()

In [ ]:
val query_3 = aux_query_3.writeStream.format("console").outputMode("update").queryName("query 3").
option("truncate", false).
trigger(Trigger.ProcessingTime("30 seconds")).start()

In [ ]:
val aux_query_4 = datos.withWatermark("ts", "20 seconds").
groupBy(window($"ts", "45 seconds", "15 seconds"), $"sensor").count()

In [ ]:
val query_4 = aux_query_4.writeStream.format("console").outputMode("append").queryName("query 4").
option("truncate", false).
trigger(Trigger.ProcessingTime("30 seconds")).start()

In [ ]:
val query_5 = datos.writeStream.format("console").
outputMode("append").queryName("query 5").
trigger(Trigger.ProcessingTime("30 seconds")).start()

In [ ]:
query_2.stop

In [ ]:
query_3.stop

In [ ]:
query_4.stop

In [ ]:
query_5.stop